### Import the required libraries for service bus and ML model

In [0]:
import asyncio
from azure.servicebus.aio import ServiceBusClient
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, datetime
import re
import time
from transformers import pipeline


### Fetching the pre-trained model for sentiment analysis

In [0]:
model_path_1="finiteautomata/bertweet-base-sentiment-analysis"
sentiment_analysis = pipeline("sentiment-analysis", model=model_path_1)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at finiteautomata/bertweet-base-sentiment-analysis.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


### Create a function to save the processed data into the delta lake

In [0]:
def save_record(results):
    resultCols=['labels','message','score']
    results_data=spark.createDataFrame(data=results, schema = resultCols)
    resultant=results_data.withColumn("timestamp",current_timestamp())
    resultant.write.mode("append").saveAsTable("sentiment_analysis_result")

### Connection the messages from the Azure Service Bus receiver end

In [0]:


#Parameters to have the connection for the Azure Service Bus 

conn_parameters=spark.read.json('/gitdemo/parameters/lparameters.json')

NAMESPACE_CONNECTION_STR=conn_parameters.collect()[0]['conn_string']
SUBSCRIPTION_NAME=conn_parameters.collect()[0]['subscription']
TOPIC_NAME=conn_parameters.collect()[0]['topic']

### Fetching the messages from the Azure Service Bus receiver end

In [0]:
async def run():
    # create a Service Bus client using the credential
    async with ServiceBusClient.from_connection_string(
        conn_str=NAMESPACE_CONNECTION_STR,
        logging_enable=True) as servicebus_client:

        async with servicebus_client:
            # get the Subscription Receiver object for the subscription
            receiver = servicebus_client.get_subscription_receiver(topic_name=TOPIC_NAME, 
            subscription_name=SUBSCRIPTION_NAME, max_wait_time=5)
            message_list=[]
            async with receiver:
                received_msgs = await receiver.receive_messages(max_wait_time=20, max_message_count=50)
                for msg in received_msgs:
                    print("Received: " + str(msg))
                    #message_list.append(json_object) 
                    message_list.append(str(msg)) 
                    # complete the message so that the message is removed from the subscription
                    await receiver.complete_message(msg)
                # call the sentiment analysis model
                results=sentiment_analysis(message_list)
                for bs_msg in range(len(results)):
                    results[bs_msg]['message']=message_list[bs_msg]
                # save the record into the delta table
                if len(results)==0:
                    print("No messages received")
                else:
                    save_record(results)  

#### Continuous the fetching of messages in a regular interval

In [0]:

#Initiate the call to the service bus from notebook
while True:
   await run()
   time.sleep(5)
   print("Time:",datetime.now())

no record
Time: 2023-05-04 07:22:27.898739
no record
Time: 2023-05-04 07:22:53.936407
no record
Time: 2023-05-04 07:23:19.971654
no record
Time: 2023-05-04 07:23:46.011410
no record
Time: 2023-05-04 07:24:12.049947
no record
Time: 2023-05-04 07:24:38.100385
no record
Time: 2023-05-04 07:25:04.134922
no record
Time: 2023-05-04 07:25:30.174101
no record
Time: 2023-05-04 07:25:56.220407
Received: hello, this is point
Time: 2023-05-04 07:26:25.302523
Received: This is not so good 
Time: 2023-05-04 07:26:33.309206
no record
Time: 2023-05-04 07:26:59.344417
Received: This is so good 
Time: 2023-05-04 07:27:24.720918
no record
Time: 2023-05-04 07:27:50.765468
no record
Time: 2023-05-04 07:28:16.813432
no record
Time: 2023-05-04 07:28:42.859864
no record
Time: 2023-05-04 07:29:08.892403
no record
Time: 2023-05-04 07:29:34.937268
no record
Time: 2023-05-04 07:30:00.978426
no record
